In [3]:
import os
import sys
import numpy as np
import json

sys.path.append('..')
from configs.arguments import get_config_dict
from utils.multiview_utils import Camera, Calibration, MultiviewVids
from utils.io_utils import write_json, load_json
from utils.metadata_utils import get_cam_names
from utils.coordinate_utils import update_reconstruction, point_in_polygon, project_to_ground_plane_cv2
from utils.plot_utils import rotation_matrix, perp

from scipy.spatial.transform import Rotation as R
from skspatial.objects import Point, Vector, Plane, Points, Line
import pyransac3d as pyrsc
from hloc.utils.read_write_model import read_points3D_binary
from hloc.utils import viz_3d
from pathlib import Path
import pycolmap


import copy
import cv2
import ipywidgets as widgets
import ipympl
import matplotlib.pyplot as plt
%matplotlib widget

arv_copy = sys.argv
sys.argv = ['pop']
sys.argv.append('-cfg')
config_path = os.path.abspath('../../project_config.yaml')
sys.argv.append(config_path)
sys.argv.append('-dr')
root_path = os.path.abspath('../../data/')
sys.argv.append(root_path)
sys.argv.append('-l')
sys.argv.append('info')

# load arguments from the arg parser
config = get_config_dict()
data_root = config["main"]["data_root"]
calib_dir = os.path.join(data_root, '0-calibration', 'calibs')
video_dir = os.path.join(data_root, 'raw_data', 'footage')
reconstruction_dir = os.path.join(data_root, '0-calibration', 'opensfm', 'undistorted', 'reconstruction.json')
omni_tag = '360'

15:13:27.253 - [...calibration/../configs/arguments.py:093] - DEBUG    - No training config file specified, using default config in data_root/2-training/train_config.yaml


In [4]:
images = Path(data_root) / "0-calibration" / "images"
outputs = Path(data_root) / "outputs"
omni_tag = "360"
reconstruction_path = outputs / 'sfm'

In [31]:
reconstruction = pycolmap.Reconstruction(outputs / 'sfm' / 'models' / '3')

In [32]:
reconstruction


Reconstruction(num_reg_images=7, num_cameras=7, num_points3D=407, num_observations=1549)

In [33]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(
    fig, reconstruction, color="rgba(255,0,0,0.5)", name="mapping", points_rgb=True
)
fig.show()

In [16]:
reconstruction.cameras

{142: Camera(camera_id=142, model=SIMPLE_RADIAL, width=512, height=512, params=[125.822005, 256.000000, 256.000000, -0.002743] (f, cx, cy, k)),
 86: Camera(camera_id=86, model=SIMPLE_RADIAL, width=512, height=512, params=[336.519307, 256.000000, 256.000000, 0.163306] (f, cx, cy, k)),
 93: Camera(camera_id=93, model=SIMPLE_RADIAL, width=512, height=512, params=[547.878633, 256.000000, 256.000000, 0.114214] (f, cx, cy, k)),
 80: Camera(camera_id=80, model=SIMPLE_RADIAL, width=512, height=512, params=[366.107425, 256.000000, 256.000000, 0.123585] (f, cx, cy, k)),
 67: Camera(camera_id=67, model=SIMPLE_RADIAL, width=512, height=512, params=[355.782825, 256.000000, 256.000000, -0.063747] (f, cx, cy, k)),
 74: Camera(camera_id=74, model=SIMPLE_RADIAL, width=512, height=512, params=[352.383940, 256.000000, 256.000000, 0.059910] (f, cx, cy, k)),
 61: Camera(camera_id=61, model=SIMPLE_RADIAL, width=512, height=512, params=[320.949178, 256.000000, 256.000000, 0.019221] (f, cx, cy, k)),
 57: Came